In [12]:
import tensorflow as tf
import keras.backend as K

In [13]:
import os
import numpy as np
from IPython import display
from PIL import Image
from glob import glob

filenames = []
labels = []

train_dir = '/home/ifranco/tf_real_estate_images_classification/data/processed/vision_based_dataset/train/'

for d in os.listdir(train_dir):
    category_filenames = glob(f'{train_dir}/{d}/*')
    filenames.extend(category_filenames)
    labels.extend([d for x in range(len(category_filenames))])

mapping = {l: idx for idx, l in enumerate(np.unique(labels))}

category = [mapping[l] for l in labels]

category = tf.keras.utils.to_categorical(category, len(np.unique(labels)))

# step 1
filenames = tf.constant(filenames)
labels = tf.constant(category)
'''
# step 2: create a dataset returning slices of `filenames`
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

# step 3: parse every image in the dataset using `map`
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    # resized_images = tf.image.resize_images(images, (img_size, img_size))
    resized_image = tf.image.resize_images(image, (img_size, img_size))
    # resized_image = tf.reshape(
    #     resized_image,
    #     (img_size * img_size * 3, ))
    return resized_image/255, label

dataset = dataset.map(_parse_function)
dataset = dataset.shuffle(5000, reshuffle_each_iteration=True).repeat(100).batch(batch_size)
'''

'\n# step 2: create a dataset returning slices of `filenames`\ndataset = tf.data.Dataset.from_tensor_slices((filenames, labels))\n\n# step 3: parse every image in the dataset using `map`\ndef _parse_function(filename, label):\n    image_string = tf.read_file(filename)\n    image_decoded = tf.image.decode_jpeg(image_string, channels=3)\n    image = tf.cast(image_decoded, tf.float32)\n    # resized_images = tf.image.resize_images(images, (img_size, img_size))\n    resized_image = tf.image.resize_images(image, (img_size, img_size))\n    # resized_image = tf.reshape(\n    #     resized_image,\n    #     (img_size * img_size * 3, ))\n    return resized_image/255, label\n\ndataset = dataset.map(_parse_function)\ndataset = dataset.shuffle(5000, reshuffle_each_iteration=True).repeat(100).batch(batch_size)\n'

In [14]:
img_size = 128
num_classes = 6

In [15]:
def tfdata_generator(images, labels, batch_size=128, shuffle=True,):
    def map_func(filename, label):
        '''A transformation function'''
        image_string = tf.read_file(filename)
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.cast(image_decoded, tf.float32)
        resized_image = tf.image.resize_images(image, (img_size, img_size))
        
        #x_train = tf.reshape(tf.cast(image, tf.float32), image_shape)
        #y_train = tf.one_hot(tf.cast(label, tf.uint8), num_classes)
        #return [x_train, y_train]
        return resized_image/255, label

    dataset  = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset  = dataset.map(map_func)
    dataset  = dataset.shuffle(5000).batch(batch_size).repeat(100)
    iterator = dataset.make_one_shot_iterator()

    next_batch = iterator.get_next()
    while True:
        yield K.get_session().run(next_batch)

In [16]:
from src.models import base_model, cnn_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

In [17]:
paths = data.PATH()

dataset_path = f'{paths.PROCESSED_DATA_PATH}/REI-Dataset/'

batch_size = 64
n_classes = 6
epochs = 100
#img_size = 256
n_channels = 3

model = cnn_model.CNN_Model(batch_size, n_classes, epochs, img_size, n_channels)

In [18]:

train_generator, validation_generator = model.get_image_data_generator(dataset_path, train=True, validation=True)
weights = model.get_class_weights(train_generator.classes, model)
#model.compile(loss=custom_losses.weighted_categorical_crossentropy(weights), metrics=['categorical_accuracy'],)
model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'],)

model.show_summary()

Found 4671 images belonging to 6 classes.
Found 562 images belonging to 6 classes.
##### CNN_Model #####
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 124, 124, 64)      4864      
_________________________________________________________________
batch_normalization_5 (Batch (None, 124, 124, 64)      256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 120, 120, 64)      102464    
_________________________________________________________________
batch_normalization_6 (Batch (None, 120, 120, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 60, 60, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 60, 60, 64)        0         
_______________

In [19]:
steps_per_epoch, validation_steps = len(train_generator.classes), len(validation_generator.classes)

In [20]:
weights

array([1.31949153, 1.24162679, 0.61687797, 1.09957627, 0.973125  ,
       1.13815789])

In [21]:
from datetime import datetime  as dt

In [22]:
init = tf.group(tf.global_variables_initializer(),
                tf.local_variables_initializer())
sess = K.get_session()
sess.run(init)

start_training = dt.now()
model.model_hist = model.model.fit_generator(
    tfdata_generator(filenames, labels),
    steps_per_epoch=steps_per_epoch,
    epochs=model.epochs,
    #validation_data=validation_generator,
    #validation_steps=validation_steps_per_epoch,
    class_weight=weights,
    #callbacks=callbacks if callbacks else None,
    workers=0,
    #max_queue_size=64,
    #use_multiprocessing=True,
).history
model.training_time = (dt.now() - start_training).total_seconds()

Epoch 1/100
 297/4671 [>.............................] - ETA: 1:00:16 - loss: 17.3349 - categorical_accuracy: 0.8473

KeyboardInterrupt: 

In [12]:
model.fit_from_generator??

In [ ]:
model.fit_from_generator(path=dataset_path,
                         train_steps_per_epoch=steps_per_epoch, validation_steps_per_epoch=validation_steps, 
                         train_generator=train_generator, validation_generator=validation_generator,
                         evaluate_net=True, use_model_check_point=False, use_early_stop=True, weighted=True)